Train GRU Tip

    - scaled data : using standarized 、 minmaxize 

    - descise the prediction of lag by using correlation coefficient 

    - the computeation time of the model is so slow , we can use dropout . we need compare dropout 

    - if model have good performation at the training data , we consider using validate data 

    -  save model with h5 fromat 

    - convert model , tensorflowjs_converter --input_format=keras c:/model.h5 c:/model  

      

In [ ]:
import pandas as pd

timestep = 10 

timestep_lag_list = []
target_list = []

for i in range(len(data)):
    
    timestep_lag_list.append(data[:i+timestep,:])

    target_list.append(data[i+timestep,:])
    
    entire_data = pd.concat([pd.DataFrame(timestep_lag_list), pd.DataFrame(target_list)], axis=1)


# Create a DataFrame from the list of lists
time_data = pd.DataFrame(entire_data, columns=[f"time_lag_{i+1}" for i in range(timestep)] + ['targettime'])


X = time_data.iloc[:, :-1].values
y = time_data.iloc[:, -1].values


# sp
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)   
 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensrorflow import keras



# Reshape the data to 3D for GRU
input= keras.Input(shape=(X_train.shape[1], 1))

layer1 = GRU(64)(input)
layer2 = Dropout(0.2)(layer1)
layer3 = GRU(64)(layer1)
layer4 = Dropout(0.2)(layer1)
output = Dense(32, activation='relu')(layer1)
output = Dense(1, activation='linear')(output)

model = keras.Model(inputs=input, outputs=output)


model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

In [ ]:
import matplotlib.pyplot as plt

loss = history.history["mae"]
val_loss = history.history["val_mae"]
epochs = range(1, len(loss) + 1)

plt.figure()
plt.plot(epochs, loss, "bo", label="Training MAE")
plt.plot(epochs, val_loss, "b", label="Validation MAE")
plt.title("Training and validation MAE")
plt.legend()
plt.show()